In [1]:
import pandas as pd 
from pathlib import Path
import alpaca_trade_api as tradeapi
import os 
from dotenv import load_dotenv
load_dotenv()
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca_trade_api.rest import REST, TimeFrame
import matplotlib.pyplot as plt
import seaborn as sns
from bear_class import BearMarket, YieldProcessor
#keys:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [2]:
sp500_df = pd.read_csv(Path('../Bear-Market-Analysis/Resources/SPX 500 1972 --_ Present - Sheet1.csv'),
 infer_datetime_format= True, parse_dates=True)
sp500_df['Date'] = pd.to_datetime(sp500_df['Date']).dt.date
sp500_df = sp500_df.set_index('Date')
sp500_df = sp500_df.dropna()

#Now importing our 10yr yield data (ST Louis Fed):
ten_yr_yield_df = pd.read_csv(Path('../Bear-Market-Analysis//Resources/DGS10 (1).csv'), infer_datetime_format=True, index_col='DATE')
ten_yr_yield_df = ten_yr_yield_df.rename(columns={'DGS10':'yield'})
ten_yr_yield_df = ten_yr_yield_df.dropna()
ten_yr_yield_df.index.names = ['Date']

#2yr yield
two_yr_yield_df = pd.read_csv(Path('../Bear-Market-Analysis//Resources/DGS2 (1).csv'), infer_datetime_format=True, index_col = 'DATE')
two_yr_yield_df = two_yr_yield_df.rename(columns={'DGS2':'yield'})
two_yr_yield_df.dropna()
two_yr_yield_df.index.names = ['Date']

#because some responses have '.' in it for the yields, we must remove them to perform .pctchg():
ten_yr_yield_df = ten_yr_yield_df[ten_yr_yield_df['yield'] != '.']
two_yr_yield_df = two_yr_yield_df[two_yr_yield_df['yield'] != '.']
#change value to float 
two_yr_yield_df=two_yr_yield_df.apply(pd.to_numeric, errors='coerce')
ten_yr_yield_df=ten_yr_yield_df.apply(pd.to_numeric, errors='coerce')

In [39]:
# Assuming sp500_df is defined elsewhere
processor = BearMarket(sp500_df)
processor.process_periods(['73_74', '80_82', '2000', '2008', '2020', '2022'])

bear_73_74, returns_73_74 = processor.get_data('73_74')
bear_80_82, returns_80_82 = processor.get_data('80_82')
bear_2000, returns_2000 = processor.get_data('2000')
bear_2008, returns_2008 = processor.get_data('2008')
bear_2020, returns_2020 = processor.get_data('2020')
bear_2022, returns_2022 = processor.get_data('2022')



In [40]:
# Assuming ten_yr_yield_df is defined elsewhere
processor = YieldProcessor(ten_yr_yield_df)
processor.process_periods(['73_74', '80_82', '2000', '2008', '2020', '2022'])

yield_73_74, yreturns_73_74 = processor.get_data('73_74')
yield_80_82, yreturns_80_82 = processor.get_data('80_82')
yield_2000, yreturns_2000 = processor.get_data('2000')
yield_2008, yreturns_2008 = processor.get_data('2008')
yield_2020, yreturns_2020 = processor.get_data('2020')
yield_2022, yreturns_2022 = processor.get_data('2022')


In [51]:
# List of labels and dataframes
labels = ['73_74', '80_82', '2000', '2008', '2020', '2022']
bear_returns_list = [returns_73_74, returns_80_82, returns_2000, returns_2008, returns_2020, returns_2022]
yield_returns_list = [yreturns_73_74, yreturns_80_82, yreturns_2000, yreturns_2008, yreturns_2020, yreturns_2022]

# Dictionary to store correlations
correlation_dict = {}

# Calculate correlations
# Calculate correlations
for label, bear_ret, yield_ret in zip(labels, bear_returns_list, yield_returns_list):
    correlation = bear_ret.values.corr(yield_ret.values)
    correlation_dict[label] = correlation

# Convert dictionary to DataFrame
correlation_df = pd.DataFrame(list(correlation_dict.items()), columns=['Year_Period', 'Correlation'])
correlation_df.set_index('Year_Period', inplace=True)
print(correlation_df)


AttributeError: 'numpy.ndarray' object has no attribute 'corr'